# Py03 - Documents Store

Documents in GDN are JSON objects. These objects can be nested (to any depth) and may contain lists. Each document has a unique primary key that identifies it within its collection. Furthermore, each document is uniquely identified by its document handle across all collections. Different revisions of the same document (identified by its handle) can be distinguished by their document revision. Any transaction only ever sees a single revision of a document.

For example:

In [ ]:
{
  "_id" : "myusers/3456789",
  "_key" : "3456789",
  "_rev" : "14253647",
  "firstName" : "John",
  "lastName" : "Doe",
  "address" : {
    "street" : "Road To Nowhere 1",
    "city" : "Gotham"
  },
  "hobbies" : [
    {name: "swimming", howFavorite: 10},
    {name: "biking", howFavorite: 6},
    {name: "programming", howFavorite: 4}
  ]
}

All documents contain special attributes:

- the document handle is stored as a string in `_id`
- the document's primary key in `_key`
- the document revision in `_rev`

The user can specify the value of the `_key` attribute when creating a document. `_id` and `_key` values are immutable once the document has been created. The `_rev` value is maintained by GDN automatically.

## Pre-requisite

Let's assume your 

- Tenant name is an email address
- User password is xxxxx

If you need to install pyC8, you can run the cell below; otherwise, you may skip it.

In [ ]:
!pip install pyC8

In [ ]:
# TIP Run this Cell so that you can output any C8QL results in JSON Format!
import json

## 1. Connect to GDN

In [ ]:
fed_url = input("Please type the federation host, leave it blank to use the default and press enter:") or "gdn.paas.macrometa.io"

In [ ]:
email = input("Please, type your email and press enter:")

In [ ]:
import getpass

password = getpass.getpass("Please, type your password and press enter:")

In [ ]:
from c8 import C8Client

client = C8Client(protocol='https', host=fed_url, port=443,
                        email=email, password=password,
                        geofabric='_system')

## 2. Get GeoFabric Details

To get details of fabric:

In [ ]:
# you might not need this, but if you wanted to select a 
# specific GeoFabric you can find out whats available by executing this code!

print("Get geo fabric details...")

# -- use this line of code for an unformatted response --
#print(client.get_fabric_details())

# -- use these lines of code for a formatted response (easier to read) -- 
fabrics = client.get_fabric_details()

print(json.dumps(fabrics, indent=4))

## 3. Create Collection

We can now create a collection in the fabric. First, you connect to fabric and then create a collection called employees.

The below example shows the steps:

In [ ]:
collection_name = 'employees'

# Create a new collection if it does not exist
if client.has_collection(collection_name):
    print("Collection exists")
else:
    client.create_collection(name=collection_name)
    print("Collection Created!")

## 4. Create Index

Let's add a `hash_index` called emails to our collection employees. Please refer to the reference guide for details on other available index types.

In [ ]:
# Simple Approach

#print("Add Hash Index", client.add_hash_index('employees', fields=['firstname', 'lastname'], unique=True)
#)                      

print("Add Hash Index")

hash_index = client.add_hash_index('employees', fields=['firstname', 'lastname'], unique=True)

print(json.dumps(hash_index, indent=4))

## 5. Insert Documents

Let's insert documents into the employees collection as shown below:

In [ ]:
#client.insert_document(collection_name='employees', document={'_key':'Alan', 'firstname': 'Alan', 'lastname':'Evans', 'email':'email'})

docs = [
  {'_key':'James', 'firstname': 'James', 'lastname':'Kirk', 'email':'email'},
  {'_key':'Han', 'firstname': 'Han', 'lastname':'Solo', 'email':'email'},
  {'_key':'Bruce', 'firstname': 'Bruce', 'lastname':'Wayne', 'email':'email'}
]

client.insert_document(collection_name='employees', document=docs)

print(json.dumps(docs, indent=4))

## 6. Query documents using C8QL

C8QL is C8's query language. You can execute C8QL query on our newly created collection of employees to get its contents.

The query `FOR employee IN employees RETURN employee` is equivalent to SQL's SELECT query.

In [ ]:
cursor = client.execute_query('FOR employee IN employees RETURN employee')

docs = [document for document in cursor]

print(json.dumps(docs, indent=4))

## 7. Get realtime updates

Example for real-time updates from a collection in fabric:

> Note: You need to enable Stream from the Macrometa Dashboard for this to work. Do this by opening up the dashboard, selecting collections, finding the collection named "employees" and selecting "enable Stream"

Now run the cell below, return to the Macrometa Dashboard and modify and save fields in the collection. You should see the real-time updates below as soon as you hit save.

The WebSocket will time out after 60 seconds if no updates are seen, so you might need to re-run the cell code if you see a timed-out message below.

In [ ]:
def callback_fn(event):
    print(event)

client.on_change("employees", callback=callback_fn)     

## 8. Time to tidy up!

That was great! Now let's tidy up by removing the collection we created.

In [ ]:
print("Collection Deleted: ", client.delete_collection("employees"))

## Section Completed!

Congratulations! You've completed this tutorial.